In [20]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split

In [21]:
data = pd.read_csv('example_data\dummy_data_processed.csv')
data.head()

# split data into X features and y labels
X = np.array(data.drop(labels='Sales', axis=1))
y = np.array(data['Sales'])

# Split the data into training and testing sets
data_train, data_test, target_train, target_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# TODO: Implement ridge regression, NN regression, gradient tree boosting and gaussian regression

**Ridge regression**

In [22]:


base_ridge = linear_model.Ridge(alpha=0.5)
calibrated_ridge = CalibratedClassifierCV(base_ridge, cv=5)
calibrated_ridge.fit(data_train, target_train)

InvalidParameterError: The 'estimator' parameter of CalibratedClassifierCV must be an object implementing 'fit' and 'predict_proba', an object implementing 'fit' and 'decision_function' or None. Got Ridge(alpha=0.5) instead.